In [14]:
# MST

import heapq
import math

def prims_mst(subgraph, nodes):
    if not nodes:
        return 0

    start = next(iter(nodes))
    visited = {start}
    min_edges = [(w, start, nb) for nb, w in subgraph[start] if nb in nodes]
    heapq.heapify(min_edges)
    cost = 0

    while min_edges and len(visited) < len(nodes):
        w, u, v = heapq.heappop(min_edges)
        if v in visited:
            continue
        visited.add(v)
        cost += w
        for nb, wt in subgraph[v]:
            if nb in nodes and nb not in visited:
                heapq.heappush(min_edges, (wt, v, nb))

    return cost if len(visited) == len(nodes) else math.inf


def tsp_with_astar(graph, start_node):
    all_nodes = list(graph.keys())
    node_index = {v: i for i, v in enumerate(all_nodes)}
    n = len(all_nodes)

    def mask(v):
        return 1 << node_index[v]

    all_mask = (1 << n) - 1
    start_mask = mask(start_node)

    frontier = [(0, 0, start_node, start_mask, [start_node])]
    best_cost = {}

    while frontier:
        f, g, u, state, path = heapq.heappop(frontier)

        if state == all_mask:
            for nb, w in graph[u]:
                if nb == start_node:
                    return path + [start_node]
            continue

        if (u, state) in best_cost and best_cost[(u, state)] <= g:
            continue
        best_cost[(u, state)] = g

        for nb, w in graph[u]:
            if state & mask(nb):
                continue
            new_state = state | mask(nb)
            new_g = g + w
            remaining = {v for v in all_nodes if not (new_state & mask(v))}
            heuristic = prims_mst(graph, remaining | {start_node, nb})
            heapq.heappush(frontier, (new_g + heuristic, new_g, nb, new_state, path + [nb]))

    return None


graph = {
    'A': [('B', 2), ('C', 9), ('D', 10)],
    'B': [('A', 2), ('C', 6), ('D', 4)],
    'C': [('A', 9), ('B', 6), ('D', 3)],
    'D': [('A', 10), ('B', 4), ('C', 3)]
}

cycle = tsp_with_astar(graph, 'A')

if cycle:
    total_cost = 0
    for i in range(len(cycle) - 1):
        u, v = cycle[i], cycle[i + 1]
        for nb, w in graph[u]:
            if nb == v:
                total_cost += w
                break
    print("Hamiltonian Cycle:", cycle)
    print("Total Cost:", total_cost)
else:
    print("No valid tour found")


Hamiltonian Cycle: ['A', 'B', 'D', 'C', 'A']
Total Cost: 18


In [16]:
import math

def tsp_branch_bound(graph, start):
    n = len(graph)
    best = {"cost": math.inf, "path": []}

    min_edge = [min(w for _, w in graph[v]) for v in range(n)]

    def dfs(path, visited, curr_cost):
        u = path[-1]

        est = curr_cost + sum(min_edge[v] for v in range(n) if v not in visited)
        if est >= best["cost"]:
            return

        if len(path) == n:
            for nb, w in graph[u]:
                if nb == start:
                    total = curr_cost + w
                    if total < best["cost"]:
                        best["cost"] = total
                        best["path"] = path + [start]
            return

        for nb, w in graph[u]:
            if nb not in visited:
                dfs(path + [nb], visited | {nb}, curr_cost + w)

    dfs([start], {start}, 0)
    return (best["path"], best["cost"]) if best["cost"] < math.inf else (None, math.inf)


graph = {
    0: [(1, 10), (2, 15), (3, 20)],
    1: [(0, 10), (2, 35), (3, 25)],
    2: [(0, 15), (1, 35), (3, 30)],
    3: [(0, 20), (1, 25), (2, 30)]
}

tour, cost = tsp_branch_bound(graph, 0)

if tour:
    print("Optimal Hamiltonian Cycle:", tour)
    print("Minimum Cost:", cost)
else:
    print("No feasible cycle found")


Optimal Hamiltonian Cycle: [0, 1, 3, 2, 0]
Minimum Cost: 80
